In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv(r'C:\Users\earth\Desktop\class_folder\FinalProject\Resources\final_table.csv')
df= df.sort_values('County')
df.head()


,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks
2417,2020-06,2020,Alamance,282450.0,423.0,53.0,352.0,28.0,124.0,2235.0,350926.0,699.0,505.50,3.16,NaN,0.012366,54.0
2982,2020-01,2020,Alamance,246798.0,566.0,75.0,256.0,12.0,172.0,1953.0,288490.0,743.0,505.50,3.62,NaN,0.037269,54.0
4046,2019-02,2019,Alamance,255000.0,681.0,101.0,320.0,32.0,152.0,2181.0,286615.0,780.0,420.2,4.37,NaN,0.043235,54.0
1274,2021-06,2021,Alamance,280000.0,197.0,20.0,388.0,12.0,48.0,1892.0,411055.0,569.0,NaN,2.98,NaN,0.018182,54.0
4981,2018-05,2018,Alamance,239900.0,809.0,64.0,408.0,48.0,200.0,2069.0,273460.0,956.0,419.9,4.59,94.0,NaN,54.0


In [4]:
# Class column based on percent change from 'median_listing_price_mm' : reference:https://stackoverflow.com/questions/21702342/creating-a-new-column-based-on-if-elif-else-condition
def f(row):
    if row['median_listing_price_mm'] == 0:
        val = 'same'
    elif row['median_listing_price_mm'] > 0:
        val = 'up'
    else:
        val = 'down'
    return val

In [5]:
df['Price_Status'] = df.apply(f, axis=1)
df.head()

,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks,Price_Status
2417,2020-06,2020,Alamance,282450.0,423.0,53.0,352.0,28.0,124.0,2235.0,350926.0,699.0,505.50,3.16,NaN,0.012366,54.0,up
2982,2020-01,2020,Alamance,246798.0,566.0,75.0,256.0,12.0,172.0,1953.0,288490.0,743.0,505.50,3.62,NaN,0.037269,54.0,up
4046,2019-02,2019,Alamance,255000.0,681.0,101.0,320.0,32.0,152.0,2181.0,286615.0,780.0,420.2,4.37,NaN,0.043235,54.0,up
1274,2021-06,2021,Alamance,280000.0,197.0,20.0,388.0,12.0,48.0,1892.0,411055.0,569.0,NaN,2.98,NaN,0.018182,54.0,up
4981,2018-05,2018,Alamance,239900.0,809.0,64.0,408.0,48.0,200.0,2069.0,273460.0,956.0,419.9,4.59,94.0,NaN,54.0,down


In [30]:
# convert county names to numbers ,dictionary of county names with values
county_names = df.County.unique()
county_di = dict(zip(county_names, range(len(county_names))))
#county_di

In [7]:
# county names with #'s'
df_num = df.copy()
df_num['County'].replace(county_di, inplace=True)
df_num.head()

,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks,Price_Status
2417,2020-06,2020,0,282450.0,423.0,53.0,352.0,28.0,124.0,2235.0,350926.0,699.0,505.50,3.16,NaN,0.012366,54.0,up
2982,2020-01,2020,0,246798.0,566.0,75.0,256.0,12.0,172.0,1953.0,288490.0,743.0,505.50,3.62,NaN,0.037269,54.0,up
4046,2019-02,2019,0,255000.0,681.0,101.0,320.0,32.0,152.0,2181.0,286615.0,780.0,420.2,4.37,NaN,0.043235,54.0,up
1274,2021-06,2021,0,280000.0,197.0,20.0,388.0,12.0,48.0,1892.0,411055.0,569.0,NaN,2.98,NaN,0.018182,54.0,up
4981,2018-05,2018,0,239900.0,809.0,64.0,408.0,48.0,200.0,2069.0,273460.0,956.0,419.9,4.59,94.0,NaN,54.0,down


In [8]:
df['violent_crime_rate'] = df['violent_crime_rate'].str.replace(',','').astype(float)
df.dtypes

Date                        object
Year                         int64
County                      object
median_listing_price       float64
active_listing_count       float64
median_days_on_market      float64
new_listing_count          float64
price_increased_count      float64
price_reduced_count        float64
median_square_feet         float64
average_listing_price      float64
total_listing_count        float64
violent_crime_rate         float64
30_Year_Fixed_Rate         float64
Number_of_Schools          float64
median_listing_price_mm    float64
Number_of_Parks            float64
Price_Status                object
dtype: object

In [10]:
filter_df =df.dropna()
#filter_df["Year"].unique() recent years have NaNs.

In [11]:
filter_df.query('County=="Cherokee"')

,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks,Price_Status
4200,2018-12,2018,Cherokee,199000.0,545.0,132.0,48.0,0.0,52.0,1708.0,264228.0,547.0,272.1,4.64,26.0,-0.004502,2.0,down
4556,2018-09,2018,Cherokee,200000.0,630.0,98.0,108.0,0.0,84.0,1783.0,265680.0,635.0,272.1,4.63,26.0,-0.069551,2.0,down
4433,2018-10,2018,Cherokee,199900.0,628.0,112.0,120.0,4.0,96.0,1750.0,262052.0,634.0,272.1,4.83,26.0,-0.000500,2.0,down
4361,2018-11,2018,Cherokee,199900.0,569.0,119.0,84.0,0.0,84.0,1766.0,266630.0,571.0,272.1,4.87,26.0,0.000000,2.0,same
4660,2018-08,2018,Cherokee,214950.0,656.0,92.0,140.0,8.0,92.0,1789.0,270440.0,660.0,272.1,4.55,26.0,-0.016022,2.0,down


In [12]:
filter_df["Year"].unique()

array([2018], dtype=int64)

In [13]:
# leave 'median_listing_price' in columns? Price_Status is classifier for median_listing_price (increased(up)/decreased(down))
columns = [
    "Date", "Year", "County", "median_listing_price",
    "active_listing_count", "median_days_on_market", "new_listing_count",
    "price_increased_count", "price_reduced_count", "median_square_feet",
    "total_listing_count", "violent_crime_rate","Number_of_Parks",
    "Number_of_Schools","30_Year_Fixed_Rate", 
     "Price_Status"]

# conditional column 'Price_status' is condition on 'median_listing...mm' which is "the percentage change
# in the median listing price from the previous month." An increase is 'up', decrease is 'down'.

# df['Price_Status'] = np.where(df['median_listing_price_mm']>=0, 'up', 'down')


# https://www.realtor.com/research/data/

target = ["Price_Status"]

In [14]:
# Load the data

df_data = filter_df.loc[:, columns].copy()
df_data = df_data.dropna()

df_data.reset_index(inplace=True, drop=True)

df_data.head()
df_data.isnull().sum()

Date                     0
Year                     0
County                   0
median_listing_price     0
active_listing_count     0
median_days_on_market    0
new_listing_count        0
price_increased_count    0
price_reduced_count      0
median_square_feet       0
total_listing_count      0
violent_crime_rate       0
Number_of_Parks          0
Number_of_Schools        0
30_Year_Fixed_Rate       0
Price_Status             0
dtype: int64

In [15]:
df_data.query('County=="Cherokee"')

,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,total_listing_count,violent_crime_rate,Number_of_Parks,Number_of_Schools,30_Year_Fixed_Rate,Price_Status
55,2018-12,2018,Cherokee,199000.0,545.0,132.0,48.0,0.0,52.0,1708.0,547.0,272.1,2.0,26.0,4.64,down
56,2018-09,2018,Cherokee,200000.0,630.0,98.0,108.0,0.0,84.0,1783.0,635.0,272.1,2.0,26.0,4.63,down
57,2018-10,2018,Cherokee,199900.0,628.0,112.0,120.0,4.0,96.0,1750.0,634.0,272.1,2.0,26.0,4.83,down
58,2018-11,2018,Cherokee,199900.0,569.0,119.0,84.0,0.0,84.0,1766.0,571.0,272.1,2.0,26.0,4.87,same
59,2018-08,2018,Cherokee,214950.0,656.0,92.0,140.0,8.0,92.0,1789.0,660.0,272.1,2.0,26.0,4.55,down


Split the Data into Training and Testing

In [16]:
# Create our features
X = df_data.drop('Price_Status', axis=1)

X = pd.get_dummies(X)

# Create our target
y = df_data.loc[:, target].copy()



In [17]:
X.describe()


,Year,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,total_listing_count,violent_crime_rate,...,County_Surry,County_Transylvania,County_Union,County_Vance,County_Wake,County_Watauga,County_Wayne,County_Wilkes,County_Wilson,County_Yadkin
count,335.0,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,...,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000,335.000000
mean,2018.0,253234.026866,637.597015,96.611940,171.892537,18.805970,179.558209,2099.773134,829.047761,298.086567,...,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925
std,0.0,95028.269661,711.972214,27.362041,275.296575,44.647468,305.507052,326.023560,966.525274,174.557044,...,0.121436,0.121436,0.121436,0.121436,0.121436,0.121436,0.121436,0.121436,0.121436,0.121436
min,2018.0,79900.000000,34.000000,18.000000,4.000000,0.000000,0.000000,1531.000000,39.000000,62.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.0,179900.000000,233.000000,77.500000,40.000000,0.000000,36.000000,1872.000000,289.000000,172.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2018.0,238000.000000,398.000000,93.000000,84.000000,4.000000,88.000000,2062.000000,498.000000,244.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2018.0,316187.500000,776.500000,114.000000,194.000000,20.000000,184.000000,2269.500000,977.000000,383.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2018.0,549000.000000,4208.000000,208.000000,1836.000000,360.000000,2128.000000,3356.000000,5361.000000,805.100000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
# Check the balance of our target values
y.value_counts()

Price_Status
down            179
up              133
same             23
dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                  stratify=y)
X_train.shape


(251, 85)

Balanced Random Forest Classifier¶

In [27]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=700, random_state=1) 
rf_model = rf_model.fit(X_train, y_train)
Counter(y_train['Price_Status'])

Counter({'down': 134, 'up': 100, 'same': 17})

In [28]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = rf_model.predict(X_test)
confusion_matrix(y_test, y_pred)


array([[11,  6, 28],
       [ 1,  2,  3],
       [ 6,  8, 19]], dtype=int64)

In [29]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)



0.3845117845117845

In [23]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        down       0.70      0.36      0.47        45
        same       0.12      0.33      0.17         6
          up       0.41      0.55      0.47        33

    accuracy                           0.43        84
   macro avg       0.41      0.41      0.37        84
weighted avg       0.54      0.43      0.45        84



In [ ]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)





In [ ]:
# heat map of feature correlation
plt.rcParams['figure.figsize']=35,35
g = sns.heatmap(df.corr(),annot=True, fmt = ".1f")

In [ ]:
# distribution plot of column
sns.distplot(df['median_listing_price'])

In [ ]:
# convert Price_class to numeric, distribution of median_listing_price increase|decrease month/month
df['Price_Status'] = df['Price_Status'].replace(['up','down'],['1','0'])
sns.distplot(df['Price_Status'])

In [ ]:
# Reference code for heatmaps :https://www.kaggle.com/code/bsivavenu/house-price-calculation-methods-for-beginners

# most correlated features
corrmat = df.corr()
top_corr_features = corrmat.index[abs(corrmat["median_listing_price"])>0.30]
plt.figure(figsize=(10,10))
g = sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="mako")

In [ ]:
# seaborn linear regression fit: median listing price vs median square feet
sns.jointplot(data=df, x="median_listing_price", y="median_square_feet", kind="reg")

# outliers present

In [ ]:
#median_listing_price vs average listing price
sns.jointplot(data=df, x="median_listing_price", y="average_listing_price", kind="reg")

In [ ]:
#median listing price vs Year
sns.jointplot(data=df, x="median_listing_price", y="Year", kind="reg")

In [ ]:
# Find Missing Ratio of Dataset
# reference: https://stackoverflow.com/questions/51070985/find-out-the-percentage-of-missing-values-in-each-column-in-the-given-dataset
#percent_missing = df.isnull().sum() * 100 / len(df)
#missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 #'percent_missing': percent_missing})
#missing_value_df

In [ ]:
# reference
# https://www.kaggle.com/code/kanncaa1/machine-learning-tutorial-for-beginners/notebook
#https://www.kaggle.com/code/erick5/predicting-house-prices-with-machine-learning/notebook